In [ ]:
from helpers_resnet import *
from torchsummary1 import summary
from torch.autograd import Variable
from sklearn.metrics import precision_score,f1_score, accuracy_score, jaccard_score

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (18,8)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

In [ ]:
resnet = torchvision.models.resnet50(pretrained=True)
adaptive_pooling = AdaptiveConcatPool2d()
head = Head()
resnet.avgpool = adaptive_pooling
resnet.fc = head

In [ ]:
resnet = resnet.cuda()

In [ ]:
df_train = pd.read_csv('./train-valid-splits-video/train.csv')
df_valid = pd.read_csv('./train-valid-splits-video/valid.csv')

In [ ]:
valid_loader = load_data(df_valid, 1, 16)

In [ ]:
show_batch(valid_loader, 1)

In [ ]:
#train_loader = load_data(df_train, 10, 1)

In [ ]:
#show_batch(train_loader, 10)

In [ ]:
for param in resnet.parameters():
    param.requires_grad = False

In [ ]:
resnet.eval()
running_loss = 0.0
running_acc = 0.0  
running_f1 = 0.0
valid_result = []
with torch.no_grad():
    for X, y in valid_loader:
        X = X.cuda()
        y = Variable(y.float()).cuda()
        X = X.squeeze(dim=0)
        y = y.squeeze(dim=1)
        y = y.float()
        output = resnet(X)
        preds = torch.sigmoid(output) #.data > 0.5  
        preds = preds.to(torch.float32)  
        preds = preds.detach().cpu().numpy()
        cl1 = []
        cl2 = []
        cl3 = []
        cl4 = []
        cl5 = []
        for p in preds:
            cl1.append(p[0])
            cl2.append(p[1])
            cl3.append(p[2])
            cl4.append(p[3])
            cl5.append(p[4])

        if (np.mean(cl1) > 0.5):
            rolling_preds[0] = 1
        else:
            rolling_preds[0] = 0

        if (np.mean(cl2) > 0.7):
            rolling_preds[1] = 1
        else:
            rolling_preds[1] = 0   

        if (np.mean(cl3) > 0.3):
            rolling_preds[2] = 1
        else:
            rolling_preds[2] = 0

        if (np.mean(cl4) > 0.8):
            rolling_preds[3] = 1
        else:
            rolling_preds[3] = 0 

        if (np.mean(cl5) > 0.8):
            rolling_preds[4] = 1
        else:
            rolling_preds[4] = 0        

        running_acc += accuracy_score(y.detach().cpu().numpy(), rolling_preds) *  X.size(0)
        running_f1 += f1_score(y.detach().cpu().numpy(), rolling_preds), average="samples")  *  X.size(0)


epoch_acc = running_acc / len(valid_loader.dataset)
epoch_f1 = running_f1 / len(valid_loader.dataset)

valid_result.append('Validation Acc: {:.4f} F1: {:.4f}'.format(epoch_acc, epoch_f1))
print(valid_result)

In [ ]:
X.shape

In [ ]:
t = y.detach().cpu().numpy(); t.shape

In [ ]:
output.shape

In [ ]:
preds = output.sigmoid()

In [ ]:
preds = preds.to(torch.float32)  

In [ ]:
preds = preds.detach().cpu().numpy()

In [ ]:
preds

In [ ]:
preds[0][1]

In [ ]:
cl1 = []
cl2 = []
cl3 = []
cl4 = []
cl5 = []
for p in preds:
    cl1.append(p[0])
    cl2.append(p[1])
    cl3.append(p[2])
    cl4.append(p[3])
    cl5.append(p[4])
    
if (np.mean(cl1) > 0.5):
    rolling_preds[0] = 1
else:
    rolling_preds[0] = 0
    
if (np.mean(cl2) > 0.5):
    rolling_preds[1] = 1
else:
    rolling_preds[1] = 0   
    
if (np.mean(cl3) > 0.5):
    rolling_preds[2] = 1
else:
    rolling_preds[2] = 0
    
if (np.mean(cl4) > 0.5):
    rolling_preds[3] = 1
else:
    rolling_preds[3] = 0 
    
if (np.mean(cl5) > 0.5):
    rolling_preds[4] = 1
else:
    rolling_preds[4] = 0

